![](https://news.dna3.com.mx/wp-content/uploads/2020/12/cenace-logo.png)
# Hyperparameter Optimization

En esta libreta se integra lo necesario para realizar la optimización de hiperparametros con Optuna y la librería neuralForecast. Registrando todo en MLFlow

### Importando bibliotecas necesarias

### Probando NeuralForecast

In [0]:
!pip install NeuralForecast

  Using cached neuralforecast-1.1.0-py3-none-any.whl (85 kB)
  Using cached numpy-1.23.5-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.1 MB)
  Using cached pandas-1.5.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.2 MB)
  Using cached ray-2.0.1-cp39-cp39-manylinux2014_x86_64.whl (60.2 MB)
  Using cached pytorch_lightning-1.6.5-py3-none-any.whl (585 kB)
  Using cached rich-12.6.0-py3-none-any.whl (237 kB)
  Using cached typing_extensions-4.4.0-py3-none-any.whl (26 kB)
  Using cached pyDeprecate-0.3.2-py3-none-any.whl (10 kB)
  Using cached torchmetrics-0.11.0-py3-none-any.whl (512 kB)
  Using cached msgpack-1.0.4-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (322 kB)
  Using cached grpcio-1.43.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (4.1 MB)
  Using cached aiosignal-1.3.1-py3-none-any.whl (7.6 kB)
  Using cached frozenlist-1.3.3-cp39-cp39-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (15

In [0]:
!pip install optuna

  Using cached optuna-3.0.4-py3-none-any.whl (348 kB)
  Using cached cmaes-0.9.0-py3-none-any.whl (23 kB)
  Using cached alembic-1.8.1-py3-none-any.whl (209 kB)
  Using cached SQLAlchemy-1.4.44-cp39-cp39-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.6 MB)
  Using cached cliff-4.1.0-py3-none-any.whl (81 kB)
  Using cached colorlog-6.7.0-py2.py3-none-any.whl (11 kB)
  Using cached numpy-1.22.4-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.8 MB)
  Using cached greenlet-2.0.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (535 kB)
  Using cached cmd2-2.4.2-py3-none-any.whl (147 kB)
  Using cached prettytable-3.5.0-py3-none-any.whl (26 kB)
  Using cached autopage-0.5.1-py3-none-any.whl (29 kB)
  Using cached stevedore-4.1.1-py3-none-any.whl (50 kB)
  Using cached pyperclip-1.8.2-py3-none-any.whl
  Using cached pbr-5.11.0-py2.py3-none-any.whl (112 kB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.

In [0]:
# Importamos librerias necesarias
import numpy as np
import pandas as pd
from neuralforecast.utils import AirPassengersDF
from IPython.display import display, Markdown
import matplotlib.pyplot as plt

from neuralforecast import NeuralForecast
from neuralforecast.models import LSTM, NHITS, RNN, GRU, TFT, DilatedRNN
from neuralforecast.auto import AutoLSTM
from neuralforecast.losses.pytorch import MAPE

import plotly.express as px
import plotly.graph_objects as go

import optuna
from pprint import pformat

import mlflow

In [0]:
df =  pd.read_csv('https://raw.githubusercontent.com/carlosvelv/temp/main/DatasetHMO_01012016_29092022.csv')
df.head()
#Data cleaning
df.drop_duplicates(subset = ['Date'], inplace=True)
df['Date'] = pd.to_datetime(df['Date'])
df = df.query('Date < 20220801')
df = df[df.Demand != 0].copy()
df.set_index('Date', inplace = True)
df = df.asfreq('H', method='pad')
df['Mes'] = df.index.month
df['Semana'] = df.index.weekofyear
df['Dia'] = df.index.weekday
df['Hora'] = df.index.hour
df.reset_index( inplace=True)

#Columnas a usar
cols = ['Date','Demand', 'Temperature', 'Semana', 'Dia', 'Hora']

df = df[cols].copy()
df.fillna(df.mean(), inplace=True)
df.head()

,Date,Demand,Temperature,Semana,Dia,Hora
0,2016-01-02 00:00:00,225.622,12.1,53,5,0
1,2016-01-02 01:00:00,213.620,11.2,53,5,1
2,2016-01-02 02:00:00,207.360,10.3,53,5,2
3,2016-01-02 03:00:00,201.374,9.4,53,5,3
4,2016-01-02 04:00:00,199.018,9.2,53,5,4


In [0]:
df.rename({'Date':'ds', 'Demand':'y'}, axis=1, inplace=True)
df['unique_id'] = 1.0
df

,ds,y,Temperature,Semana,Dia,Hora,unique_id
0,2016-01-02 00:00:00,225.622,12.1,53,5,0,1.0
1,2016-01-02 01:00:00,213.620,11.2,53,5,1,1.0
2,2016-01-02 02:00:00,207.360,10.3,53,5,2,1.0
3,2016-01-02 03:00:00,201.374,9.4,53,5,3,1.0
4,2016-01-02 04:00:00,199.018,9.2,53,5,4,1.0
...,...,...,...,...,...,...,...
57667,2022-07-31 19:00:00,610.326,24.7,30,6,19,1.0
57668,2022-07-31 20:00:00,609.114,26.4,30,6,20,1.0
57669,2022-07-31 21:00:00,610.594,26.3,30,6,21,1.0
57670,2022-07-31 22:00:00,619.377,26.6,30,6,22,1.0


In [0]:
train_df = df.query('ds < 20220601')
test_df = df.query('ds >= 20220601')

In [0]:
# Obtain hyperparameters for this trial
def suggest_hyperparameters(trial):
    # Obtain the learning rate on a logarithmic scale
    max_epochs = trial.suggest_categorical("max_epochs", [200,300,400,500, 600])
    #Scaler
    scaler = trial.suggest_categorical("scaler", ['standard'])
    #lr
    lr = trial.suggest_float("lr", 1e-4, 1e-1, log=True)
    # Obtain the dropout ratio in a range from 0.0 to 0.9 with step size 0.1
    #input_size = trial.suggest_float("dropout", 0.2, 0.9, step=0.1)
    # Obtain the optimizer to use by name
    #optimizer_name = trial.suggest_categorical("optimizer_name", ["Adam", "Adadelta", "RMSprop", "SGD"])
    
    #Encoder decoder params
    encoder_n_layers = trial.suggest_categorical('encoder_n_layers', [2,3,4,5])
    
    encoder_hidden_size = trial.suggest_categorical('encoder_hidden_size', [200,250,300,350])
    
    encoder_activation = trial.suggest_categorical('encoder_activation', ['tanh', 'relu'])
    
    context_size = trial.suggest_categorical('context_size', [10,20,30,40,50])
    
    decoder_hidden_size = trial.suggest_categorical('decoder_hidden_size', [200,250,300,350])

    print(f"Suggested hyperparameters: \n{pformat(trial.params)}")
    return max_epochs, scaler, lr, encoder_n_layers, encoder_hidden_size, encoder_activation, context_size, decoder_hidden_size

In [0]:
def objective(trial):
    print("\n********************************\n")
    best_val_loss = float('Inf')
    
    #Fixed params
    selected_attr = ['Demand', 'Temperature', 'Semana', 'Dia', 'Hora'] 
    nombre_corrida = 'Pruebas para RNN'
    experiment_path = "/carlos.velazquez.unison@cenace.gob.mx/LSTM-HMO/hyperparam_opt/benchmarking"
    horizon = 24
    input_size = -1
    
    

    experiment = mlflow.set_experiment(experiment_path)
    with mlflow.start_run(run_name=nombre_corrida) as run:

        
        max_epochs, scaler, lr, encoder_n_layers, encoder_hidden_size, encoder_activation, context_size, decoder_hidden_size = suggest_hyperparameters(trial)
    
        
        mlflow.log_params(trial.params)
        
        ### Loggeando Parametros ###
        mlflow.log_param('model', 'RNN')
        mlflow.log_param('horizon', horizon)
        mlflow.log_param('input_size', input_size)
        mlflow.log_param('attr', selected_attr)    


        ### Generando serie de entrenamiento ###
        train_df = df.query('ds < 20220601')
        test_df = df.query('ds >= 20220601')

        #### Generando el modelo
        models = [RNN(h=horizon, input_size=input_size,
                #loss=MAE(),
                loss=MAPE(),
                #loss=DistributionLoss(distribution='Normal', level=[80, 90]),
                scaler_type='standard',
                encoder_n_layers=encoder_n_layers,
                encoder_hidden_size=encoder_hidden_size,
                context_size=context_size,
                decoder_hidden_size=decoder_hidden_size,
                decoder_layers=2,
                max_epochs=max_epochs,
                hist_exog_list=['Temperature', 'Semana', 'Dia', 'Hora'])]


        fcst = NeuralForecast(models=models, freq='H')
        fcst.fit(df=train_df, )
        forecasts = fcst.predict()
        
        forecasts.set_index('ds', inplace=True)
        plot_df = forecasts.join(test_df.set_index('ds').y)

        

        fig = go.Figure()
        fig.add_trace(go.Scatter(x=plot_df.index, y=plot_df.RNN, name="Estimada"))
        fig.add_trace(go.Scatter(x=plot_df.index, y=plot_df.y, name="Real"))
        fig.update_layout(title="Estimación de la demanda")


        #Logeando grafica
        mlflow.log_figure(fig, "real_vs_estimado.html")

        actual = plot_df.y.values
        pred = plot_df.RNN.values
        ape = np.abs((actual - pred) / actual)
        mape = np.mean(ape) * 100
        
        mlflow.log_metric('MAPE', mape)


        mlflow.end_run()
    
    return mape

In [0]:
def main():
    # Create the optuna study which shares the experiment name
    study = optuna.create_study(study_name="mlflow-optuna", direction="minimize")
    study.optimize(objective, n_trials=5)

    # Print optuna study statistics
    print("\n++++++++++++++++++++++++++++++++++\n")
    print("Study statistics: ")
    print("  Number of finished trials: ", len(study.trials))

    print("Best trial:")
    trial = study.best_trial

    print("  Trial number: ", trial.number)
    print("  Loss (trial value): ", trial.value)

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

In [0]:
%%capture
main()

[I 2022-12-07 00:30:08,526] A new study created in memory with name: mlflow-optuna
[I 2022-12-07 01:15:32,380] Trial 0 finished with value: 16.40240137853695 and parameters: {'max_epochs': 200, 'scaler': 'standard', 'lr': 0.02331549532163362, 'encoder_n_layers': 4, 'encoder_hidden_size': 350, 'encoder_activation': 'tanh', 'context_size': 10, 'decoder_hidden_size': 350}. Best is trial 0 with value: 16.40240137853695.
[I 2022-12-07 01:39:25,716] Trial 1 finished with value: 10.189826050358358 and parameters: {'max_epochs': 200, 'scaler': 'standard', 'lr': 0.00013562861047263175, 'encoder_n_layers': 3, 'encoder_hidden_size': 350, 'encoder_activation': 'tanh', 'context_size': 20, 'decoder_hidden_size': 350}. Best is trial 1 with value: 10.189826050358358.
[I 2022-12-07 01:43:46,195] Trial 2 finished with value: 4.348296275876475 and parameters: {'max_epochs': 300, 'scaler': 'standard', 'lr': 0.052817254520869744, 'encoder_n_layers': 3, 'encoder_hidden_size': 200, 'encoder_activation': 'tan